In [2]:
import keras.applications as app
from keras.models import Model  
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 
from keras import backend as K  
# 返回模型以及数据预处理方法
def get_model(name='vgg16'):
    if name == 'vgg16':
        model = app.vgg16.VGG16(weights=r'C:\Users\tomding\vgg16_weights_tf_dim_ordering_tf_kernels.h5')
        preprocess_func = app.vgg16.preprocess_input
    if name == 'vgg19':
        model = app.vgg19.VGG19(weights='imagenet')
        preprocess_func = app.vgg19.preprocess_input
    if name == 'resnet50':
        model = app.resnet50.ResNet50(weights='imagenet')
        preprocess_func = app.resnet50.preprocess_input
    if name == 'inception_v3':
        model = app.inception_v3.InceptionV3(weights='imagenet')
        preprocess_func = app.inception_v3.preprocess_input
    if name == 'xception':
        model = app.xception.Xception(weights='imagenet')
        preprocess_func = app.xception.preprocess_input
    if name == 'mobilenet':
        model = app.mobilenet.MobileNet(weights='imagenet')
        preprocess_func = app.mobilenet.preprocess_input
    if name == 'densenet':
        model = app.densenet.DenseNet121(weights='imagenet')
        preprocess_func = app.densenet.preprocess_input

    return model, preprocess_func


Using TensorFlow backend.


In [3]:
# 读取并根据传入的方法处理图片，然后返回原图处理后的图片
def read_img(img_path, preprocess_func, size):
    img = cv2.imread(img_path)
    processed_img = cv2.resize(img, size)

    processed_img = np.expand_dims(processed_img, axis=0)
    processed_img = preprocess_func(processed_img)

    return img, processed_img

In [4]:
# 把原始输出图像重构为我们可以看懂得三通道图像
def deprocess_image(origin_img):
    # 标准化张量: center 为 0., 保证 std 为 0.25
    origin_img = (origin_img-origin_img.mean()) / (origin_img.std()+K.epsilon()) * 0.25

    # clip to [0, 1] 后面两个参数分别表示最小和最大值
    origin_img = np.clip(origin_img+0.5, 0, 1)

    # convert to RGB array
    origin_img *= 255
    if K.image_data_format() == 'channels_first':
          origin_img = origin_img.transpose((1, 2, 0))
    img = np.clip(origin_img, 0, 255).astype('uint8')
    return img

In [5]:
# 对一个张量用其L2模进行归一化，避免特别小或特别大的梯度，保证梯度上升过程中梯度平滑
def normalize(x):     
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [6]:
# 可视化卷积输出或过滤器，输入原始图像，行列数，类型，保存图片名称
def vis_conv_filter(images, col_row_num, layer_name, kind):
    # 显示多少个图像
    row_num = col_row_num
    col_num = col_row_num
    # 图像大小
    size = images[0].shape[1]

    fig, axes = plt. subplots(nrows=row_num, ncols=col_num, figsize=(16, 8))
    for i in range(row_num):
        for j in range(col_num):
            if kind == 'filter':
                img = images[i+j*col_num]
            if kind == 'conv':
                img = images[:, :, i+j*col_num]
            img = cv2.resize(img, (size, size))
            axes[i, j].imshow(img)
            # 不显示刻度
            axes[i, j].set_xticks([])
            axes[i, j].set_yticks([])

    plt.savefig(r'visualize_images\{}\{}.jpg'.format(kind, layer_name), dpi=600)
    plt.show()

In [7]:
# 可视化热力图，输入原始图像及热图
def vis_heatmap(img, heatmap):
    #img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure()
    # 原图
    plt.subplot(221)
    plt.imshow(cv2.resize(img, (224, 224)))
    plt.axis('off')
    # 热力图
    plt.subplot(222)
    plt.imshow(heatmap)
    plt.axis('off')
    # 叠加
    plt.subplot(212)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    # 将热力图转换为RGB格式
    heatmap = np.uint8(255 * heatmap)

    # 将热力图应用于原始图像
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    # 这里的0.4是热力图强度因子
    superimposed_img = heatmap*0.4 + img
    plt.imshow(superimposed_img)
    plt.axis('off')
    # 现实的是错误的但是cv保存正确
    cv2.imwrite('heatmap.jpg', superimposed_img)

    plt.tight_layout()
    plt.savefig(r'visualize_images\heatmap.jpg', dpi=600)
    plt.show()

In [8]:
# 获取某卷积层的输出 (28, 28, 512)
def conv_output(model, layer_name, img):

    # 输入图象placeholder
    input_img = model.input
    # 根据传入的名称在字典里找
    try:
        out_conv = model.get_layer(layer_name).output
    except:
        raise Exception('Not layer named {}!'.format(layer_name))

    # 截断模型
    intermediate_layer_model = Model(inputs=input_img, outputs=out_conv)
    # 得到中间结果
    intermediate_output = intermediate_layer_model.predict(img)

    return intermediate_output[0]

In [9]:
# 获取模型某层卷积过滤核感兴趣的图像返回每个卷积核loss最大的图像
# 随机初始化一张输入图像，然后求得某个卷积核输出对图像的梯度，更改图像，使得某个卷积核输出值最大化。
# 这里需要使损失函数值“最大化”，也就是使某个卷积核激活输出的值最大，所以称为梯度上升。
def conv_filter(model, filter_num, layer_name, img):

    # 输入图象placeholder
    input_img = model.input

    # 每层名字和层的字典不包含输入层
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

    # 根据传入的名称在字典里找层
    try:
        layer_output = layer_dict[layer_name].output
        # assert isinstance(output_layer, layers.Conv2D)
    except:
        raise Exception('Not layer named {}!'.format(layer_name))

    #　用于存储使每个卷积核输出最大的输入图像及损失函数值
    filtermax_i_v = []
  
    # 共有512个卷积核layer_output.shape  (?, 28, 28, 512) 
    for i in range(filter_num):
        print('computing {}th filter, still have {}'.format(i+1, filter_num-i-1))
        # 建立目标损关函数，我们的优化目标就是最大化指定某层索引为i的卷积核的激活值
        # 因为是最大化目标损失函数，而不是最小化，所以成为“梯度上升”
        if K.image_data_format() == 'channels_first':             
            loss = K.mean(layer_output[:, i, :, :])         
        else:             
            loss = K.mean(layer_output[:, :, :, i])
        # 计算目标损关函数对图像的梯度
        grads = K.gradients(loss, input_img)[0]
        # 使用上面定义的normalize函数处理梯度
        grads = normalize(grads)
        # 返回给定输入图像的损失和梯度
        # loss.shape, grads.shape () (?, 224, 224, 3)
        iterate = K.function([input_img], [loss, grads])

        # 梯度上升的步长
        step = 1.
    
        fimg = img.copy()

        # 运行梯度上升20个步
        for j in range(20):
            loss_value, grads_value = iterate([fimg])
            # 梯度乘以步长更改
            fimg += grads_value * step

        # 由此产生的输入图像进行解码
        fimg = deprocess_image(fimg[0])
        # plt.imshow(fimg)  
        # plt.show()
        filtermax_i_v.append((fimg, loss_value))

    # 根据loss从大到小排序过滤器结果
    filtermax_i_v.sort(key=lambda x: x[1], reverse=True)
    # 返回每个卷积核loss最大的图像
    return np.array([f[0] for f in filtermax_i_v])

In [10]:
# 获取某层卷积图像的热力图
def heatmap_output(model, last_conv_layer_name, processed_img):

    # 对图像的预测类别
    preds = model.predict(processed_img)
    # 该类别对应的索引
    index = np.argmax(preds[0])
    # 预测向量中是该类的元素  model.output.shape (1, 1000) 
    class_output = model.output[:, index]
    # 输出特征图   last_conv_layer.output（1，7，7，512）
    last_conv_layer = model.get_layer(last_conv_layer_name)

    # 该类别类对最后一层卷积输出特征图的梯度 返回的是只有一个元素的（1，7，7，512）张量列表
    grads = K.gradients(class_output, last_conv_layer.output)[0]

    # 平均特定特征图通道上的梯度
    pooled_grads = K.mean(grads, axis=(0, 1, 2))

    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([processed_img])

    for i in range(conv_layer_output_value.shape[-1]):
        # 将特征图数组的每个通道乘以这个通道对该类别重要程度
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    # 得到的特征图的逐通道的平均值即为类激活的热力图
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    # heatmap与0比较，取其大者
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap

In [ ]:
img_path = r'cat.jpg'
model, preprocess_func = get_model('vgg16')
img, processed_img = read_img(img_path, preprocess_func, (224, 224))
plt.figure()
plt.subplot(131)
plt.imshow(img)
# opencv的接口使用BGR，而matplotlib.pyplot 则是RGB模式
img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.subplot(132)
plt.imshow(img_RGB)
plt.subplot(133)
plt.imshow(processed_img[0])
plt.show()

In [ ]:
layer_name = 'block4_conv3'
row_col = 4   # 显示row_col**2个图像
# 卷积输出可视化,将处理好的图片送到模型中提取出并画出某层的输出
conv_out = conv_output(model, layer_name, processed_img)
vis_conv_filter(conv_out, row_col, layer_name, 'conv')
# 第一层卷积层类似边缘检测的功能，在这个阶段里，卷积核基本保留图像所有信息 
# 随着层数的加深，卷积核输出的内容也越来越抽象，保留的信息也越来越少。
# 越深的层数，越多空白的内容，也就说这些内容空白卷积核没有在输入图像中找到它们想要的特征

In [ ]:
# 卷积核到底是如何识别物体的呢？想要解决这个问题，有一个方法就是去了解卷积核最感兴趣的图像是怎样的。
# 我们知道，卷积的过程就是特征提取的过程，每一个卷积核代表着一种特征。如果图像中某块区域与某个卷积核
# 的结果越大，那么该区域就越“像”该卷积核。
# 基于以上的推论，如果我们找到一张图像，能够使得这张图像对某个卷积核的输出最大，那么我们就说找到了该
# 卷积核最感兴趣的图像。
# 具体思路：输入一张随机内容的图像II, 求某个卷积核FF对图像的梯度 G=∂F/∂IG=∂F/∂I，用梯度上升的方法迭
# 代更新图像 I=I+η∗GI=I+η∗G，ηη是类似于学习率的东西。
layer_name = 'block5_conv1'
row_col = 6   # 显示row_col**2个图像
filter_num = 36
# 卷积输出可视化,将处理好的图片送到模型中提取出并画出某层的输出
random_img =(np.random.random((1, 224, 224, 3))-0.5)*20 + 128
plt.imshow(deprocess_image(random_img[0]))
plt.show()
filters_out = conv_filter(model, filter_num, layer_name, random_img)
vis_conv_filter(filters_out, row_col, layer_name, 'filter')
# 低层的卷积核似乎对颜色，边缘信息感兴趣。
# 越高层的卷积核，感兴趣的内容越抽象（非常魔幻啊），也越复杂。
# 高层的卷积核感兴趣的图像越来越难通过梯度上升获得（block5_conv1有很多还是随机噪声的图像）

In [ ]:
last_conv_layer_name = 'block5_conv3'
# 在图像分类问题中，假设网络将一张图片识别成“猫”的概率是0.9，我想了解到底最后一层的卷积层 
# 对这0.9的概率的贡献是多少。换句话时候，假设最后一层卷积层有512个卷积核，我想了解这512个卷积核 
# 对该图片是”猫”分别投了几票。投票越多的卷积核，就越确信图片是“猫”，因为它们提取到的特征趋向猫的特征。
# 代码中，输入了一图片，然后获得最后一层卷积层的热度图，最后将热度图叠加到原图像，获得图像中起到关键分类作用的部分。
heatmap = heatmap_output(model, last_conv_layer_name, processed_img) 
vis_heatmap(img, heatmap)